In [1]:
import geemap
import json
import os
import requests
import re
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import pandas as pd
import ee
import numpy as np

#initialize GEE using your Google Account
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [24]:
#set up GEE map canvas (will be updated with every addLayer() method)
Map = geemap.Map(center=[-27.93186,32.48897],zoom=9)
Map.add_basemap('SATELLITE')
Map.add_basemap('CartoDB.DarkMatter')

#study area
file_path = './data/boundaries/isimangaliso_lesser_bbox.geojson'
with open(file_path) as f:
    coord = json.load(f)['features'][0]['geometry']['coordinates'][0][0]
    line_geom = ee.Geometry.LineString(coord)
    
    #polygon used for clipping purposes
    poly_geom = ee.Geometry.Polygon(coord)
    
Map.addLayer(ee_object=line_geom, vis_params={'color':'red'}, name="Lesser Isimangaliso Wetland Park")
Map
    


Map(center=[-27.93186, 32.48897], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [3]:
def get_meta_s2(data_info):
    prod_id = data_info['properties']['PRODUCT_ID']
    date = pd.to_datetime(re.findall(r"(\d{8})T", prod_id)[0]).date()
    cl_cover = round(data_info['properties']['CLOUD_COVERAGE_ASSESSMENT'],2)
    tile = re.findall(r"(\d{2}[A-Z]{3})", prod_id)[0]
    df = pd.DataFrame({'id':prod_id,'date':date,'tile':tile,'cloudcover':cl_cover},index=[0])
    return df

def filter_duplicates (s2_col_list):
    s2_filtered = []
    for i in range(s2_col.size().getInfo()):
        prod_id =  s2_col_list.get(i).getInfo()['properties']['PRODUCT_ID']
        if 'OPER_PRD' not in prod_id:
            s2_filtered += [ee.Image(s2_col_list.get(i))]
    s2_filtered_col = ee.ImageCollection(s2_filtered)
    s2_filtered_list = s2_filtered_col.toList(s2_filtered_col.size())
    return s2_filtered_list

In [4]:
#get Sentinel-2 Level-1C collection for 2016
s2_col = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(line_geom).filterDate('2016-01-01','2016-12-31') \
    .filter(ee.Filter.inList('MGRS_TILE', ['36JVP','36JVQ'])) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 0)) \
    .select(['B2', 'B3', 'B4','B8','B8A','B11','B12'])\


#filter duplicated products 
s2_col_list = filter_duplicates(s2_col.toList(s2_col.size()))

print('Nr. images found: ', s2_col_list.size().getInfo())

#store product information
products = list(map(get_meta_s2,s2_col_list.getInfo()))
products_df = pd.concat(products).reset_index(drop=True)


Nr. images found:  10


In [5]:
#find matching tiles by date
tile_matches = {}
for i in range(s2_col_list.size().getInfo()):
    data = s2_col_list.get(i)
    prod_id = data.getInfo()['properties']['PRODUCT_ID']
    date = str(pd.to_datetime(re.findall(r"(\d{8})T", prod_id)[0]).date())
    tile_matches.setdefault(date,[]).append(ee.Image(data))
    

In [20]:
#create a mosaic from the matched tiles
s2_mosaics = []
for date in tile_matches:
    match = tile_matches[date]
    if len(match)==2:
        img_mos = ee.ImageCollection(match).mosaic().clip(poly_geom)
        s2_mosaics += [img_mos.set({'Date': date})]
    else:
        img = match[0].clip(poly_geom)
        s2_mosaics += [img.set({'Date': date})]
s2_mos_col = ee.ImageCollection(s2_mosaics)
s2_mos_col_list = s2_mos_col.toList(s2_mos_col.size())

In [25]:
# #visualize RGB mosaics
# for i in range(s2_mos_col_list.size().getInfo()):
#     data = s2_mos_col_list.get(i)
#     img_mos = ee.Image(data)
#     date = data.getInfo()['properties']['Date']
#     viz_rgb = {'bands': ['B4', 'B3', 'B2'],'gain': [0.1, 0.1, 0.1],'scale':30}
#     Map.addLayer(img_mos, viz_rgb, date)
    
data = s2_mos_col_list.get(-1)
img_mos = ee.Image(data)
date = data.getInfo()['properties']['Date']
viz_rgb = {'bands': ['B4', 'B3', 'B2'],'gain': [0.1, 0.1, 0.1],'scale':30}
Map.addLayer(img_mos, viz_rgb, date)

In [26]:
def add_wiw(img):
    wiw = img.expression(
        '((B8A/10000) <= 0.1804) && ((B12/10000) <= 0.1131)',{'B8A':img.select('B8A'),'B12':img.select('B12')}
    ).rename('WIW')
    return img.addBands(wiw)

def add_rws(img):
    mndwi = img.normalizedDifference(['B3','B11']).rename('MNDWI')
    mgrn = img.select(['B4','B3','B8']).reduce(ee.Reducer.min()).rename('MGRN')
    rws = img.expression(
        'MNDWI > 0.3 && (MGRN/10000) < 0.15',{'MNDWI':mndwi,'MGRN':mgrn}
    ).rename('RWS')
    return img.addBands(rws)


In [22]:
s2_mos_stack = s2_mos_col.map(add_rws)
s2_mos_stack = s2_mos_stack.map(add_wiw)
s2_mos_stack_list  = s2_mos_stack.toList(s2_mos_stack.size())

In [28]:
wiw = ee.Image(s2_mos_stack_list.get(-1)).select('WIW')
viz = {'min': 1, 'max': 1, 'palette': ['blue'],'scale':30}
new_wiw = wiw.updateMask(wiw.eq(1))
Map.addLayer(new_wiw, {'palette':'blue'}, 'WIW 2016-09-01')

rws = ee.Image(s2_mos_stack_list.get(-1)).select('RWS')
new_rws = rws.updateMask(rws.eq(1))
Map.addLayer(new_rws, {'palette':'blue'}, 'RWS 2016-09-01')